# skills 过程演示（离线 mock）
展示 `skill catalog -> match -> selection -> execution plan -> final_answer`。

In [ ]:
from component import load_skill_catalog, match_skill_scores, select_skill, run_demo

In [ ]:
user_text = "请帮我创建一个新技能，用于规范化代码评审流程。"
user_text

In [ ]:
catalog = load_skill_catalog()
catalog

In [ ]:
scores = match_skill_scores(user_text, catalog)
scores

In [ ]:
selection = select_skill(user_text, catalog)
selection

In [ ]:
demo_result = run_demo(user_text)
demo_result["final_answer"]

In [ ]:
demo_result["trace"]